# Deploy the Fastai Classifier using Gradio and Hugging Face Spaces

# Preparations

In [4]:
# connect colab to google drive by mounting
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# navigate to WasteWise directory
%cd drive/MyDrive/wt23-wastewise/

[Errno 2] No such file or directory: 'drive/MyDrive/wt23-wastewise/'
/content/drive/MyDrive/wt23-wastewise


In [ ]:
# get latest version of branch
!git pull

remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 20 (delta 9), reused 15 (delta 8), pack-reused 0
Unpacking objects: 100% (20/20), 6.57 MiB | 1.50 MiB/s, done.
From https://github.com/TechLabs-Berlin/wt23-wastewise
   ce6fda1..4d13a02  MLOps                   -> origin/MLOps
   d16396d..0e3e4e2  classifier_fastai       -> origin/classifier_fastai
   c52b9b6..0579f18  machine-learning-Marina -> origin/machine-learning-Marina
Updating ce6fda1..4d13a02
Fast-forward
 AI/MLOps/deploy_fastai/deploy_fastai_gradio.ipynb | 482 ++++++++++++++--------
 1 file changed, 305 insertions(+), 177 deletions(-)


In [ ]:
# install the fastbook library
!pip install -Uqq fastbook

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 KB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# install gradio
!pip install -Uqq gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 KB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 KB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.1/144.1 KB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.5/129.5 KB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 KB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 6.1 MB/s eta 0:00:00


In [6]:
# import libraries
# fastbook/fastai related libraries: used for training the classifier
from fastbook import *

# import gradio for deployment 
import gradio as gr

In [ ]:
# disable warnings
import warnings
warnings.filterwarnings('ignore')

# Load the model

In [7]:
# load the model
learn = load_learner('/content/drive/MyDrive/wastewise_models/resnet101_waste_recogniser_fastai_v2.pkl', cpu=True)

In [ ]:
# test model inference on banana peel and see how long it takes
%time learn.predict("/content/drive/MyDrive/wt23-wastewise/AI/data_20_classes/banana_peels/10_175867801.jpg")

CPU times: user 342 ms, sys: 7.42 ms, total: 349 ms
Wall time: 358 ms


('banana_peels',
 tensor(2),
 tensor([3.1576e-04, 5.4867e-02, 8.5881e-01, 1.5164e-02, 7.3338e-03, 3.3237e-03, 2.3346e-04, 3.6541e-03, 2.9444e-04, 7.1263e-03, 1.9181e-03, 4.0224e-04, 2.3900e-04, 1.1081e-04, 2.4067e-03, 5.7777e-04,
         2.9609e-02, 6.4534e-04, 2.8585e-03, 1.0105e-02]))

The fine tuned network has 101 layers and I was concerned it would take too long for inference on CPU, but this is definitely reasonable.

In [8]:
# make a "list" containing the classes the classifier can distinguish
# note that actual data type is not "list", but "fastai.data.transforms.CategoryMap"
# it still works in a comparable way
labels = learn.dls.vocab

In [ ]:
# see labels
print(type(labels))
print(labels)
print("Label at index 2: " + labels[2])

<class 'fastai.data.transforms.CategoryMap'>
['aluminum_foil', 'apples', 'banana_peels', 'cardboard', 'condoms', 'diapers', 'food_waste', 'glass_bottle', 'old_books', 'oranges', 'pans', 'pizza_box', 'plastic_bags', 'plastic_packaging', 'plastic_toys', 'smartphone', 'tampons', 'tea_bags', 'tetrapack', 'toothbrush']
Label at index 2: banana_peels


# Make a gradio app that can be used in an API for the real app.

In this case, I keep the outputs minimal, because the rest will be solved in JavaScript and HTML together with our WebDev.

Just make it classify the image, nothing more.

If I include more, it will be difficult for our web dev to integrate it with his proper app.

In [9]:
# define a function for the learner
def predict(img):
    img = PILImage.create(img)
    pred,pred_idx,probs = learn.predict(img)

    return {labels[i]: float(probs[i]) for i in range(len(labels))}

In [10]:
# make a gradio interface
gr.Interface(fn=predict, inputs=gr.Image(shape=(512, 512)), outputs=gr.Label(num_top_classes=3)).launch(share=False)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

# Make a proper gradio website that can be shown off as a standalone

The previous example was very basic, but there are so many possibilities in Gradio! Now I will do a little more frontend myself and assemble an actual usable waste recognizer website with gradio.

In [ ]:
!pip install scikit-image

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import skimage

In [ ]:
# make dictionaries translating the class to something else for the output

# adapt class spelling for output
spelling_dict = {
    "aluminum_foil": "aluminum foil",
    "apples": "apple",
    "banana_peels": "banana peel",
    "cardboard": "cardboard",
    "condoms": "condom",
    "diapers": "diaper",
    "food_waste": "food waste",
    "glass_bottle": "glass bottle",
    "old_books": "book",
    "oranges": "orange",
    "pans": "pan",
    "pizza_box": "pizza box",
    "plastic_bags": "plastic bag",
    "plastic_packaging": "plastic packaging",
    "plastic_toys": "plastic toy",
    "smartphone": "smartphone",
    "tampons": "tampon",
    "tea_bags": "tea bag",
    "tetrapack": "tetra pak",
    "toothbrush": "toothbrush"
    }


    # recommend waste bin for each class
bin_dict = {
    "aluminum_foil": "gelbe sack",
    "apples": "bio waste",
    "banana_peels": "bio waste",
    "cardboard": "paper waste",
    "condoms": "residual waste",
    "diapers": "residual waste",
    "food_waste": "residual waste",
    "glass_bottle": "glass waste",
    "old_books": "paper waste",
    "oranges": "bio waste",
    "pans": "residual waste",
    "pizza_box": "residual waste",
    "plastic_bags": "plastic waste",
    "plastic_packaging": "plastic waste",
    "plastic_toys": "residual waste",
    "smartphone": "wertstoffsammlung",
    "tampons": "residual waste",
    "tea_bags": "bio waste",
    "tetrapack": "plastic waste",
    "toothbrush": "residual waste"}

In [ ]:
labels = list(spelling_dict.values())

In [ ]:
# describe the interface
title = "WasteWise"
description = "Hey! What do you want to throw away? \n Just take a picture by clicking the camera icon, then hit 'submit'. \n You will see the result and confidence to the right hand side as well as a recommendation below. \n To label the image as misclassified, just hit 'Flag'. \n To see what parts of the image are responsible for the output, hit 'Interpret' and wait for a few. \n  The parts of the image that contributed to increase the likelihood of the outputted class are marked red. \n The parts that decrease the class confidence are highlighted blue. \n The intensity of color corresponds to the importance of that part of the input. \n At the bottom you can load some example images."
examples = ['/content/drive/MyDrive/wt23-wastewise/AI/data_20_classes/banana_peels/10_175867801.jpg',
            "/content/drive/MyDrive/wt23-wastewise/AI/data_20_classes/pizza_box/31_can-you-recycle-pizza-boxes-and-know-the-rule-L-R6CB5u.jpeg",
            "/content/drive/MyDrive/wt23-wastewise/AI/data_20_classes/tea_bags/36_b3dcc3f8f7fe76facafeb7ef2cbeeb5f--the-tea-the-works.jpg",
            "/content/drive/MyDrive/wt23-wastewise/AI/data_20_classes/plastic_bags/21_recycle-plastic-bags.jpg"]

In [ ]:
# define a function for the learner
def predict_full(img):
    img = PILImage.create(img)
    pred,pred_idx,probs = learn.predict(img)

    return {labels[i]: float(probs[i]) for i in range(len(labels))}, f'It belongs into the {bin_dict[pred]}.'

In [ ]:
# make a gradio interface
gr.Interface(fn=predict_full, inputs = gr.Webcam(shape=(512, 512)), title=title, description=description, examples=examples, interpretation = "default", enable_queue = True, outputs=[gr.outputs.Label(num_top_classes=3), "text"]).launch(share=False)

/usr/local/lib/python3.9/dist-packages/gradio/outputs.py:197: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)
/usr/local/lib/python3.9/dist-packages/gradio/deprecation.py:40: UserWarning: `enable_queue` is deprecated in `Interface()`, please use it within `launch()` instead.
  warnings.warn(value)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

### A quick word about the interpretation
I added a simple CAM interpretation class activation mapping (CAM) to the standalone app. Once the image is classified, the user can can click "Interpret". The image is then dissected into squares. The resolution here is rather bad, it's not individual pixels, but square groups of them, but it's still a cool feature that's interactive and available for the user. The squares are colored according to their importance for the classification. Please note that there are other options for this available. For example, a shapley-based interpreter is supported as well and can be used by setting `interpretation = "shap` as well as adding `shap` to the requirements (or rather installing shap, when you run it locally). This allows for a more detailed interpretation, but comes with __much__ longer run rimes and I figured that user would not like to wait this long.

# Create new Hugginface Space

To make the app available online "permanently"

## app.py

This file needs to be saved to "app.py".

It consists of importing the dependencies, loading the classifier, initializing the predict function and setting up the interface.

### For the only-API app
```python
import gradio as gr
from fastai.vision.all import *
import skimage

# load image classifier
learn = load_learner('resnet101_waste_recogniser.pkl')

# get classes of waste
labels = learn.dls.vocab

# define a function for the learner
# define a function for the learner
def predict(img):
    img = PILImage.create(img)
    pred,pred_idx,probs = learn.predict(img)

    return {labels[i]: float(probs[i]) for i in range(len(labels))}

# make a gradio interface
gr.Interface(fn=predict, inputs=gr.Image(shape=(512, 512)), outputs=gr.Label(num_top_classes=3)).launch(share=False)
```

### For the standalone app
```python
import gradio as gr
from fastai.vision.all import *
import skimage

# load image classifier
learn = load_learner('resnet101_waste_recogniser.pkl')

# get classes of waste
labels = learn.dls.vocab

# make dictionaries translating the class to something else for the output
# adapt class spelling for output
spelling_dict = {
    "aluminum_foil": "aluminum foil",
    "apples": "apple",
    "banana_peels": "banana peel",
    "cardboard": "cardboard",
    "condoms": "condom",
    "diapers": "diaper",
    "food_waste": "food waste",
    "glass_bottle": "glass bottle",
    "old_books": "book",
    "oranges": "orange",
    "pans": "pan",
    "pizza_box": "pizza box",
    "plastic_bags": "plastic bag",
    "plastic_packaging": "plastic packaging",
    "plastic_toys": "plastic toy",
    "smartphone": "smartphone",
    "tampons": "tampon",
    "tea_bags": "tea bag",
    "tetrapack": "tetra pak",
    "toothbrush": "toothbrush"
    }
# recommend waste bin for each class
bin_dict = {
    "aluminum_foil": "gelbe sack",
    "apples": "bio waste",
    "banana_peels": "bio waste",
    "cardboard": "paper waste",
    "condoms": "residual waste",
    "diapers": "residual waste",
    "food_waste": "residual waste",
    "glass_bottle": "glass waste",
    "old_books": "paper waste",
    "oranges": "bio waste",
    "pans": "residual waste",
    "pizza_box": "residual waste",
    "plastic_bags": "plastic waste",
    "plastic_packaging": "plastic waste",
    "plastic_toys": "residual waste",
    "smartphone": "wertstoffsammlung",
    "tampons": "residual waste",
    "tea_bags": "bio waste",
    "tetrapack": "plastic waste",
    "toothbrush": "residual waste"}


# define a function for the learner
def predict_full(img):
    img = PILImage.create(img)
    pred,pred_idx,probs = learn.predict(img)

    return {labels[i]: float(probs[i]) for i in range(len(labels))}, f'It belongs into the {bin_dict[pred]}.'


# describe the interface
title = "WasteWise"
description = "Hey! What do you want to throw away? \n Just take a picture by clicking the camera icon, then hit 'submit'. \n You will see the result and confidence to the right hand side as well as a recommendation below. \n To label the image as misclassified, just hit 'Flag'. \n To see what parts of the image are responsible for the output, hit 'Interpret' and wait for a few seconds. \n  The parts of the image that contributed to increase the likelihood of the outputted class are marked red. \n The parts that decrease the class confidence are highlighted blue. \n The intensity of color corresponds to the importance of that part of the input. \n At the bottom you can load some example images."
# examples = [...] # enter examples and path here, save them in GitHub

# make a gradio interface
gr.Interface(fn=predict_full, inputs = gr.Webcam(shape=(512, 512)), title=title, description=description, interpretation = "default", enable_queue = True, outputs=[gr.outputs.Label(num_top_classes=3), "text"]).launch()

```
## requirements.txt

This file need to be saved to "requirements.txt".

It contains the packages the machine needs to install.
```
fastai
scikit-image
```

## Upload the files

1. Create an account at Hugging Face 🤗
2. Navigate to "Spaces" and click "Create new Space"
3. Set its name, license etc. and select "Gradio".
4. Launch
5. This will create a remote git repository. Clone it to your local.
6. Initialise git large file storage. For this, if on Mac, install Homebrew, navigate to the repo and type "git install lfs". It allows you to upload files larger than 100 MB, most importantly your trained model.
7. Move all relevant files (app.py, requirements.txt, <your_model>.pkl to that repo.
8. Add, commit, push, wait.
9. Once building is done, the app can be used.

## The deployed apps can be found here

#### Standalone version
https://huggingface.co/spaces/fabianjkrueger/WasteWise

#### API version
https://huggingface.co/spaces/fabianjkrueger/WasteWise_API

## Usage via API

1. Follow one of the links. Both can be used via an API, but the API version is much better suited for integration with our JavaScript app, hence the name.
2. Upon opening the link, you can find "Use via API" at the very bottom. Open that, then just switch to JavaScript to see the instructions.
3. Follow the instructions. Images need to be converted to base 64 strings in order to be passed. Passing the converted string to a variable, then passing the string to the request is strongly recommended.

## More cool ideas
Another feature that can be activated is to flag the images as misclassified. They then get saved and can be reviewed and reused for training. This is very powerful once the app is properly deployed and used. Currently, however, since the Huggingface Spaces are public, flagged images are visible to everyone in the internet, which comes with some serious data privacy concerns. Your personal waste is not a matter for everyone to see, which is why flagging currently is activated. But as mentioned previously, in case the app ever goes into serious production, we might implement it in combination with data privacy measure.
